### 필요한 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

### 전처리 완료한 데이터 불러오기

In [2]:
data = pd.read_csv('./data/Result3_Preprocessed.csv', low_memory = False)
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13,desc_noun
0,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.8,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN,이 땅 비밀 영문 판 산 중턱 층층 바위 사이 돌이 물고기 이 물고기 아이 물고기 ...
1,9791186259948,NaN,어쩜 좋아!,박은율 (지은이),시시한출판사,20000.0,https://image.aladin.co.kr/product/25964/92/co...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,813.8,2020-12-24,NaN,259649242,그림책 <어쩜 좋아!>는 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림책-...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,NaN,어린이 그림책 권 출판사 주최 그림책 공모전 어린이 그림책 부문 선정 작품 비밀 약...
2,9788936433444,NaN,괭이부리말 아이들,김중미 지음;송진헌 그림,창작과 비평사,12000.0,http://image.aladin.co.kr/product/30/41/cover/...,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,813.6,2001-11-07,893643344X:,304191,NaN,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,NaN,두 권 의 양장본 제 어린이 책 원고 공모 창작 부문 대상 수상작 이 작품 방송 오...
3,9788951508752,NaN,꿈꾸는 동글이,동화창의학습연구회 편;김숙경 [공]그림,동화사,7000.0,http://image.aladin.co.kr/product/29/37/cover/...,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",813.6,2001-08-24,8951508757,293751,NaN,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",NaN,새 교과서 말 듣기 쓰기 창의 학습 동화 국어 수학 과학 사회 도덕 등 모든 과목 ...
4,9788956511368,NaN,서울 여자와 대구 남자가 만날 때,샤베트 지음,영상노트,9000.0,http://image.aladin.co.kr/product/42/5/cover/8...,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,813.6,2003-06-27,8956511365,420501,정민 로맨스 소설.,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,NaN,참여정부 행정 수도 한강 남쪽 지역 것 공약 때 서울 태생 서울 작가 서울 애정 가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32318,9791189824051,5,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 5,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/18011/35/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-01-02,NaN,180113502,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32319,9791164242979,9,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 9,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/19362/34/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,813.7,2019-05-29,NaN,193623473,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32320,9791164245963,11,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 11,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20122/76/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-07-31,NaN,201227638,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32321,9791136500687,12,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 12,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20646/2/cov...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-09-03,NaN,206460246,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...


In [3]:
data_by_isbn = data.set_index(keys = data['isbn13'], drop = True)
data_by_isbn # ISBN 코드에 대응하는 도서 데이터

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13,desc_noun
isbn13,,,,,,,,,,,,,,,,
9791195991341,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.8,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN,이 땅 비밀 영문 판 산 중턱 층층 바위 사이 돌이 물고기 이 물고기 아이 물고기 ...
9791186259948,9791186259948,NaN,어쩜 좋아!,박은율 (지은이),시시한출판사,20000.0,https://image.aladin.co.kr/product/25964/92/co...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,813.8,2020-12-24,NaN,259649242,그림책 <어쩜 좋아!>는 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림책-...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,NaN,어린이 그림책 권 출판사 주최 그림책 공모전 어린이 그림책 부문 선정 작품 비밀 약...
9788936433444,9788936433444,NaN,괭이부리말 아이들,김중미 지음;송진헌 그림,창작과 비평사,12000.0,http://image.aladin.co.kr/product/30/41/cover/...,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,813.6,2001-11-07,893643344X:,304191,NaN,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,NaN,두 권 의 양장본 제 어린이 책 원고 공모 창작 부문 대상 수상작 이 작품 방송 오...
9788951508752,9788951508752,NaN,꿈꾸는 동글이,동화창의학습연구회 편;김숙경 [공]그림,동화사,7000.0,http://image.aladin.co.kr/product/29/37/cover/...,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",813.6,2001-08-24,8951508757,293751,NaN,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",NaN,새 교과서 말 듣기 쓰기 창의 학습 동화 국어 수학 과학 사회 도덕 등 모든 과목 ...
9788956511368,9788956511368,NaN,서울 여자와 대구 남자가 만날 때,샤베트 지음,영상노트,9000.0,http://image.aladin.co.kr/product/42/5/cover/8...,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,813.6,2003-06-27,8956511365,420501,정민 로맨스 소설.,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,NaN,참여정부 행정 수도 한강 남쪽 지역 것 공약 때 서울 태생 서울 작가 서울 애정 가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9791189824051,9791189824051,5,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 5,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/18011/35/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-01-02,NaN,180113502,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
9791164242979,9791164242979,9,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 9,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/19362/34/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,813.7,2019-05-29,NaN,193623473,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
9791164245963,9791164245963,11,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 11,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20122/76/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-07-31,NaN,201227638,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...


### 도서별 코사인 유사도 계산

In [4]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['desc_noun'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) # 코사인 유사도 계산 결과
isbn_to_idx = dict(zip(data['isbn13'], data.index))

In [5]:
cos_sim

array([[1.        , 0.06063244, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06063244, 1.        , 0.10596048, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.10596048, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.8543182 ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.8543182 ],
       [0.        , 0.        , 0.        , ..., 0.8543182 , 0.8543182 ,
        1.        ]])

### 호불호 표시 도서 목록

In [6]:
like, interest, hate = [], [], [] # '재밌어요'/'관심있어요'/'별로예요' 표시 도서 목록

### 동일 세트 ISBN 함수
기능: 입력 받은 ISBN 코드와 동일한 세트의 ISBN 코드 반환

In [7]:
def setIsbn(isbn: int):
    set_num = data_by_isbn.loc[isbn]['set_isbn13']
    try:
        set_num = int(set_num)
        target = data_by_isbn[data_by_isbn['set_isbn13'] == set_num]
        return list(target.index)
    except ValueError:
        return [isbn]

### 유사 도서 ISBN 함수
기능: 입력 받은 ISBN 코드의 도서와 유사도가 높은 도서 및 동일한 세트 도서의 ISBN 코드 반환<br>
목적: '별로예요' 표시 도서와 유사한 도서를 제거하기 위해

In [8]:
def similarIsbn(isbn_list: list, cos_sim = cos_sim):
    sim_isbn = []
    for isbn in isbn_list:
        idx = isbn_to_idx[isbn] # isbn 코드에 대응하는 인덱스
        sim_score = list(enumerate(cos_sim[idx]))
        sim_score.sort(key = lambda x: x[1], reverse = True) # 유사도 기준으로 내림차순 정렬
        sim_score = sim_score[:2] # 유사도 기준 상위 2개
        sim_idx = [sim[0] for sim in sim_score]
        sim_isbn += [data.iloc[i][0] for i in sim_idx]
        sim_isbn += setIsbn(isbn)
    
    return list(set(sim_isbn))

### 유사 도서 추천 함수
기능: 입력 받은 ISBN 코드의 도서와의 유사도 기준 상위 5개의 도서 ISBN 코드 반환<br>
특징: 제외할 도서가 있을 경우 유사도 결과에서 모두 제거한 후 추천 도서 반환

In [9]:
def recommendIsbn(isbn: int, cos_sim = cos_sim, key = 1):
    global like, interest, hate
    
    if key:
        idx = isbn_to_idx[isbn] # isbn 코드에 대응하는 인덱스
        del_isbn = like + interest + similarIsbn(hate) # 제외 도서 리스트: '재밌어요'/'관심있어요' 표시 도서, '별로예요' 표시 도서와 유사한 도서
        del_idx = [isbn_to_idx[i] for i in del_isbn]
        del_idx += [isbn_to_idx[i] for i in setIsbn(isbn)]
    else:
        idx = len(data) - len(similarIsbn(like) + interest + similarIsbn(hate)) - 1
        del_idx = [idx]
    
    sim_score = list(enumerate(cos_sim[idx]))
    
    for i in sorted(list(set(del_idx)), reverse = True): # 유사도 결과에서 제거
        del sim_score[i]
    
    sim_score.sort(key = lambda x: x[1], reverse = True) # 유사도 기준으로 내림차순 정렬
    sim_score = sim_score[:5]
    
    return list(list(zip(*sim_score))[0])

### '재밌어요' 도서 기반 추천 도서 제공 함수
기능: '재밌어요' 표시 도서 기반으로 유사도 기준 상위 5개의 도서 ISBN 코드 반환

In [10]:
def likeIsbn():
    global like, interest, hate
    
    reco_data = data_by_isbn.copy()[['isbn13', 'desc_noun']]
    all_desc_noun = ' '.join(reco_data.loc[like]['desc_noun'])
    
    reco_data.drop(similarIsbn(like) + interest + similarIsbn(hate), inplace = True)
    reco_data.reset_index(drop = True, inplace = True)
    reco_data = pd.concat([reco_data, pd.DataFrame({'isbn13': [0], 'desc_noun': [all_desc_noun]})], ignore_index = True)
    
    tfidf_matrix = tfidf.fit_transform(reco_data['desc_noun'])
    reco_cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) # 코사인 유사도 계산 결과
    idx_list = recommendIsbn(0, cos_sim = reco_cos_sim, key = 0)
    
    return list(reco_data.iloc[idx_list].isbn13)